The analysis began with the use of Microsoft Excel tools, specifically [Power Query](https://learn.microsoft.com/en-us/power-query/power-query-what-is-power-query) and [Power Pivot](https://support.microsoft.com/en-us/office/power-pivot-overview-and-learning-f9001958-7901-4caa-ad80-028a6d2432ed), to inspect, merge, and clean the dataset. I standardized formatting and naming conventions, addressing inconsistencies in country names, and added columns to facilitate demographic analysis, including Age Groups, Generations, Regions, and Sub-Regions. The five regions in this article are made up of the following sub-regions:
•	Africa – Northern, Western, Eastern, Southern, and Central
•	Americas – North, South, Central, the Caribbean
•	Asia – North, East, Southeast, South, Central, and West
•	Europe – Northern, Southern, Eastern, Western
•	Oceania – Australasia, Melanesia, Micronesia, and Polynesia

Subsequently, I transitioned to [Python](https://en.wikipedia.org/wiki/Python_(programming_language)) (Version 3.12) and [Jupyter Notebook](https://www.geeksforgeeks.org/data-science/jupyter-notebook/) for interactive data analysis. To visualise the data, I employed [Plotly](https://plotly.com/python/), an open-source graphing library that allows for the embedding of .html style figures. Additional Python packages used:
-	Pandas: For data manipulation and analysis
-	Numpy: For numerical operations and handling arrays
-	os: For file and directory manipulation, to interact with the operating system
-	OrderedDict from Collections: Specialised dictionary with order preservation. Useful when order of items is important
-	Math: For complex mathematical calculations without building them from scratch

I converted the dataset from .xlsx (Excel) to .csv (Comma Separated Value) format and performed the following, additional, cleaning steps:
-	Filtered out invalid values while retaining relevant data.
-	Reversed the WVS/EVS confidence scale, so that instead of 1 = highest confidence and 4 = lowest confidence it is now vice versa such that 1 = lowest confidence and 4 = highest confidence 

Finally, most of my functions implement Pandas’ pivot tables which helps summarise and aggregate data from my dataset. This allowed me to group data by specific columns (i.e., Age Groups) and perform calculations such as the mean or median (etc.) on grouped data. Main functions are found in plot_functions.ipynb.


In [8]:
import pandas as pd

def clean_data(df):
    """
    Cleans and normalizes WVS/EVS survey data.
    - Preserves valid data across all columns without dropping entire rows.
    - Filters invalid values column-wise using .where()
    - Applies reverse scale to confidence-related variables + nationalpride.
    Returns cleaned DataFrame and list of confidence columns.
    """

    # --- Ordered Categories ---
    wave_order = ["1981-'84", "1990-'94", "1995-'98", "1999-'04", "2005-'09", "2010-'14", "2017-'22"]
    age_order = ["Teen (<18)", "Young Adult (18-30)", "Middle Adult (31-45)",
                 "Older Adult (46-65)", "Elder Adult (66+)"]
    gen_order = ["Lost Generation (1883-'00)", "Greatest Generation (1901-'27)", "Silent Generation (1928-'45)",
                 "Baby Boomer (1946-'64)", "Gen X (1965-'80)", "Millennial (1981-'96)", "Gen Z (1997-'12)",
                 "DNK(-1)", "NO ANS(-2)", "N/A(-3)", "Not in EVS(-4)", "MISSING(-5)"]
    

    # --- Valid Ranges ---
    valid_sex = ["Male", "Female"]
    valid_born = ["Yes", "No"]
    valid_confidence = [-1, -2, 1, 2, 3, 4]
    valid_politicalself = [-1, -2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    valid_govself = [-1, -2, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    valid_pride = [-1, -2, 1, 2, 3, 4]
    valid_generations = gen_order[:7]

    confidence_columns = [
        'Press_Confidence', 'Police_Confidence', 'Government_Confidence',
        'PolParties_Confidence', 'EU_Confidence', 'UN_Confidence'
    ]

    

    # --- Normalize Sex and BornHere codes (stored as strings in your CSV)
    df['Sex'] = df['Sex'].replace({'1': 'Male', '2': 'Female'})
    df['BornHere'] = df['BornHere'].replace({'0': 'No', '1': 'Yes'})

    # --- Set ordered categorical types
    df['Years'] = pd.Categorical(df['Years'], categories=wave_order, ordered=True)
    df['Age Group'] = pd.Categorical(df['Age Group'], categories=age_order, ordered=True)
    df['Generation'] = pd.Categorical(df['Generation'], categories=gen_order, ordered=True)

    

    # --- Filter invalid values column-wise (preserve rows, just set invalid cells to NaN)
    df['Sex'] = df['Sex'].where(df['Sex'].isin(valid_sex))
    df['BornHere'] = df['BornHere'].where(df['BornHere'].isin(valid_born))
    df['Generation'] = df['Generation'].where(df['Generation'].isin(valid_generations))
    df['PoliticalSelfRating'] = df['PoliticalSelfRating'].where(df['PoliticalSelfRating'].isin(valid_politicalself))
    df['GovVsSelf'] = df['GovVsSelf'].where(df['GovVsSelf'].isin(valid_govself))
    df['NationalPride'] = df['NationalPride'].where(df['NationalPride'].isin(valid_pride))

     
    
    # --- Convert confidence columns to numeric (if not already)
    df[confidence_columns] = df[confidence_columns].apply(pd.to_numeric, errors='coerce')

    # --- Filter invalid confidence scores
    df[confidence_columns] = df[confidence_columns].apply(
        lambda col: col.where(col.isin(valid_confidence))
    )

    

    # --- Reverse scale for confidence values + NationalPride
    reverse_scale = {1: 4, 2: 3, 3: 2, 4: 1}
    df[confidence_columns + ['NationalPride']] = df[confidence_columns + ['NationalPride']].replace(reverse_scale)

    

    return df, confidence_columns
